In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests

# Retrieving the latest news about Mars from the Nasa website. 

In [2]:
executable_path = {"executable_path": 'chromedriver.exe'}
browser = Browser("chrome", **executable_path, headless=False)

In [3]:
url1 = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url1)

html1 = browser.html
soup1 = bs(html1, "html.parser")

In [4]:
list1 = soup1.find_all("div", class_="list_text")

In [5]:
news_title = list1[0].find("a",target="_self").text.strip()
news_p = list1[0].find("div",class_="article_teaser_body").text.strip()

In [6]:
print(f"""Title: {news_title}
Paragraph: {news_p}""")

Title: Mars InSight's 'Mole' Is Moving Again
Paragraph: The NASA lander's robotic arm seems to have helped its heat probe burrow almost 2 centimeters (3/4 of an inch) since last week.


# Retrieving the url link for the current featured image from the JPL/NASA website. 

In [7]:
url2 = "https://www.jpl.nasa.gov/spaceimages/"
browser.visit(url2)

html2 = browser.html
soup2 = bs(html2, "html.parser")

In [8]:
featured_image_url = soup2.find("div", class_="carousel_items").find("article")
featured_image_url = featured_image_url["style"].split("url('/")[1]
featured_image_url = featured_image_url.split("'")[0]
featured_image_url = "https://www.jpl.nasa.gov/"+featured_image_url
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17936-1920x1200.jpg'

# Retrieving the latest weather info from a twitter account. 

In [9]:
url3 = "https://twitter.com/marswxreport"
browser.visit(url3)

html3 = browser.html
soup3 = bs(html3, "html.parser")

In [10]:
list3  = soup3.find_all("div", class_="js-tweet-text-container")

In [11]:
mars_weather = list3[0].find("p").text.strip().replace("\n"," ").split("pic.twitter")[0]
mars_weather

'InSight sol 314 (2019-10-15) low -102.8ºC (-153.0ºF) high -24.8ºC (-12.6ºF) winds from the SSE at 4.9 m/s (11.1 mph) gusting to 19.7 m/s (44.0 mph) pressure at 7.20 hPa'

In [12]:
# # Storing the results obtained above in a dictionary.
dictionary1 = {
    "news_title": news_title,
    "news_p": news_p,
    "featured_image_url": featured_image_url,
    "mars_weather": mars_weather,
}

# Retrieving the facts table from a website. 

In [13]:
url4 = 'https://space-facts.com/mars/'

In [14]:
tables = pd.read_html(url4)
len(tables)

2

In [15]:
df = tables[1]
df.columns = ['description', 'value']
df = df.set_index("description")
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [16]:
dictionary2 = df.to_dict()["value"]

# Retrieving the url for 4 Mars images and their titles from a website.

In [17]:
url5 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url5)
html5 = browser.html
soup5 = bs(html5, "html.parser")
list5  = soup5.find("div", class_="collapsible results")

titles = []
for item in list5.find_all("a"):
    title = item.text.split(" ")[0]
    if title != "":
        titles.append(title)

hemisphere_image_urls = []
count = 0
for item in list5.find_all("a"):
    title = item("h3")
    if title != []:
        browser.click_link_by_partial_text(titles[count])
        html = browser.html
        soup = bs(html, "html.parser")
        title = soup.find("h2", class_="title").text.strip()
        image_url = soup.find("a", target="_blank")["href"]
        dictionary = {"title":title,"image_url":image_url}
        hemisphere_image_urls.append(dictionary)
        browser.back()
        count += 1
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [18]:
dictionary3 = {"image_titles":[hemisphere_image_urls[0]["title"],hemisphere_image_urls[1]["title"],
                              hemisphere_image_urls[2]["title"],hemisphere_image_urls[3]["title"]],
              "image_urls":[hemisphere_image_urls[0]["image_url"],hemisphere_image_urls[1]["image_url"],
                            hemisphere_image_urls[2]["image_url"],hemisphere_image_urls[3]["image_url"]]} 

In [19]:
mars_data = {**dictionary1, **dictionary2, **dictionary3}
mars_data

{'news_title': "Mars InSight's 'Mole' Is Moving Again",
 'news_p': "The NASA lander's robotic arm seems to have helped its heat probe burrow almost 2 centimeters (3/4 of an inch) since last week.",
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17936-1920x1200.jpg',
 'mars_weather': 'InSight sol 314 (2019-10-15) low -102.8ºC (-153.0ºF) high -24.8ºC (-12.6ºF) winds from the SSE at 4.9 m/s (11.1 mph) gusting to 19.7 m/s (44.0 mph) pressure at 7.20 hPa',
 'Equatorial Diameter:': '6,792 km',
 'Polar Diameter:': '6,752 km',
 'Mass:': '6.39 × 10^23 kg (0.11 Earths)',
 'Moons:': '2 (Phobos & Deimos)',
 'Orbit Distance:': '227,943,824 km (1.38 AU)',
 'Orbit Period:': '687 days (1.9 years)',
 'Surface Temperature:': '-87 to -5 °C',
 'First Record:': '2nd millennium BC',
 'Recorded By:': 'Egyptian astronomers',
 'image_titles': ['Cerberus Hemisphere Enhanced',
  'Schiaparelli Hemisphere Enhanced',
  'Syrtis Major Hemisphere Enhanced',
  'Valles Marineris Hemisph